In [14]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/aremath/p38-env/lib/python3.8/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['Rectangle', 'Circle', 'copy', 'interactive', 'isnan', 'Button', 'Widget', 'Enum', 'Text', 'Polygon']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
import sys
sys.path.append("..")

In [3]:
from ipyleaflet import *
from ipywidgets import Layout

In [4]:
location = ((0,0), (1,1))
m = Map(center=(0.5, 0.5), zoom=8, layout=Layout(width="500px", height="500px"))
m.clear_layers()
len(m.layers)
image = ImageOverlay(url="https://www.vgmaps.com/Atlas/SuperNES/SuperMetroid-Zebes(In-GameMap).png", bounds=location)
m.add_layer(image)
m.fit_bounds(location)

In [5]:
from ipywidgets import *

In [6]:
m

Map(center=[0.5, 0.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [48]:
circle = CircleMarker()
circle.location = (0.5, 0.5)
circle.radius = 5
circle.color="white"
circle.fill_color="white"
m.add_layer(circle)

In [16]:
import math
import time

In [22]:
# Testing moving the circle around
n = 1000
l = linspace(0, 2*math.pi, n)
y = 0.5 * np.sin(l)
x = 0.5 * np.cos(l)

In [23]:
#for i in range(n):
#    circle.location = (x[i] + 0.5, y[i] + 0.5)
#    time.sleep(0.03)

In [24]:
def mk_color(rgb):
    r,g,b = rgb
    all_str = ""
    for c in r, g, b:
        assert c >= 0 and c < 256
        c_str = hex(c)[2:]
        if len(c_str) < 2:
            c_str = f"0{c_str}"
        all_str += c_str
    return f"#{all_str}"

In [25]:
#B4D455
mk_color((0xB4, 0xD4, 0x55))

'#b4d455'

In [26]:
#from_emu_path = "..."
#to_emu_path = "..."
#from_emu = open(from_emu_path, "rb")
#to_emu = open(to_emu_path, "w+")

## Determining the map scale

In [27]:
from world_rando.coord import Coord

In [28]:
# Use global position for shared reference point
#$079F: Area index
#{
#    0: Crateria
#    1: Brinstar
#    2: Norfair
#    3: Wrecked Ship
#    4: Maridia
#    5: Tourian
#    6: Ceres
#    7: Debug
#}
area_offsets = {
    0: Coord(3, 10),
    1: Coord(0, 29),
    2: Coord(31, 49),
    3: Coord(37, 0),
    4: Coord(28, 29),
    5: Coord(0, 10),
    # Out of bounds
    6: Coord(0, -10),
    7: Coord(0, 0),
}

In [29]:
# Scale = ???

#0.98y
def position_transform(emu_p):
    cc = Coord(0.01358, 0.01594) * emu_p
    return Coord(-cc.y, cc.x) + Coord(1.110, 0)

In [30]:
area_colors = ["white", "green", "red", "teal", "blue", "pink"]

In [31]:
for i in range(len(area_colors)):
    area_start = area_offsets[i]
    area_end = area_offsets[i] + Coord(64, 32)
    area_rect = Rectangle(bounds=(position_transform(area_start), position_transform(area_end)))
    color = area_colors[i]
    area_rect.color = color
    area_rect.fill_color = color
    m.add_layer(area_rect)

In [32]:
# Area Rects
rect = Rectangle(bounds=((0.985,0.01), (0.89,0.107)))
m.add_layer(rect)

In [33]:
# Y conversion
(0.985 - 0.89) / 7

0.013571428571428568

In [34]:
# X conversion
(0.107 - 0.01) / 7

0.013857142857142858

In [35]:
# ~ 1 to 0.0135
# With starting point at...

In [5]:
from rom_tools.rom_manager import RomManager
from rom_tools.rom_data_structures import RoomHeader

In [37]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")

In [38]:
parsed = rom.parse()

In [39]:
def get_room_position(rh):
    return Coord(rh.map_x, rh.map_y) + area_offsets[rh.area_index]

In [40]:
rh = [get_room_position(o) for o in parsed.values() if isinstance(o, RoomHeader)]

In [41]:
all_room_positions = [position_transform(r) for r in rh]

In [42]:
for r in all_room_positions:
    circle = Circle()
    circle.location = r
    circle.radius = 1
    circle.color = "green"
    circle.fill_color = "green"
    m.add_layer(circle)

In [43]:
from abstraction_validation.abstractify import abstractify_state

In [135]:
def read_state():
    ram_data = np.fromfile("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16")
    return abstractify_state(ram_data, global_pos=True)

In [ ]:
#ram_data = np.memmap("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16", mode="c")#.reshape((-1, 0x2000 // 2))
while True:
    ram_data = np.fromfile("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16")
    #print(ram_data.shape)
    time.sleep(0.1)
    p = abstractify_state(ram_data, global_pos=True).position
    #print(p)
    map_tile = Coord(p.x / 16, p.y / 16)
    #print(map_tile)
    #print(position_transform(map_tile))
    circle.location = position_transform(map_tile)

In [ ]:
f = open("../../Videogame-Retrieval/ram_snes9x.bin", "rb")
len(f.read())

In [ ]:
# 3 Threads:
# - Location Display
# - Direction finding
# -> Outputs a Lua Script
# Lua script file reads the script and evals the code

In [77]:
# Testing Lua injection
lua_fn = """
local px_addr = 0x000AF6
local py_addr = 0x000AFA
local camx_addr = 0x000911
local camy_addr = 0x000915
local camx = memory.readword(camx_addr)
local camy = memory.readword(camy_addr)
local px = memory.readword(px_addr)
local py = memory.readword(py_addr)
local screen_x = px - camx
local screen_y = py - camy
local xrad = memory.readbyte(0x7E0AFE)
local yrad = memory.readbyte(0x7E0B00)	
gui.box(screen_x + (xrad * -1), screen_y + (yrad * -1), screen_x + xrad, screen_y + yrad,"#0000FF35","#0000FFFF")
gui.text(screen_x, screen_y, "{}", 0xffffffff)
gui.text(0, 20, "{}", 0xffffffff)
"""
ss = "Hello"

In [78]:
while True:
    if ss == "Hello":
        ss = "Goodbye"
    else:
        ss = "Hello"
    f = open("../../Videogame-Retrieval/snes9x_bridge.lua", "w")
    f.write(lua_fn.format(ss, ss))
    f.close()
    time.sleep(1)

KeyboardInterrupt: 

In [11]:
from abstraction_validation.sm_paths import *
from encoding.parse_rooms import parse_rooms

In [6]:
rom = RomManager("../../roms/sm_clean.sfc", "sm_junk.smc")

In [7]:
parsed_rom = rom.parse()

In [12]:
rooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")

In [13]:
all_posns = all_global_positions(rooms, parsed_rom)

Landing_Site 0x791f8
	 Extra: []
	 Missing: ['Landing_Site_Ship']
West_Ocean 0x793fe
	 Extra: ['West_Ocean_L', 'West_Ocean_R4']
	 Missing: ['West_Ocean_L2', 'West_Ocean_R6']
Bomb_Torizo 0x79804
	 Extra: ['Bomb_Torizo_B']
	 Missing: ['Bomb_Torizo_Bombs']
Spore_Spawn 0x79dc7
	 Extra: []
	 Missing: ['Spore_Spawn_Spore_Spawn']
Kraid 0x7a59f
	 Extra: []
	 Missing: ['Kraid_Kraid']
Statues 0x7a66a
	 Extra: []
	 Missing: ['Statues_Statues']
Crocomire 0x7a98d
	 Extra: []
	 Missing: ['Crocomire_Crocomire']
Water_Closet 0x7b1e5
	 Extra: []
	 Missing: ['Water_Closet_Drain']
Golden_Torizo 0x7b283
	 Extra: []
	 Missing: ['Golden_Torizo_Golden_Torizo']
Ridley 0x7b32e
	 Extra: []
	 Missing: ['Ridley_Ridley']
Phantoon 0x7cd13
	 Extra: []
	 Missing: ['Phantoon_Phantoon']
West_Sand_Hall 0x7d461
	 Extra: []
	 Missing: ['West_Sand_Hall_TS']
East_Sand_Hall 0x7d4c2
	 Extra: []
	 Missing: ['East_Sand_Hall_TS']
West_Sand_Hole 0x7d4ef
	 Extra: []
	 Missing: ['West_Sand_Hole_TS']
East_Sand_Hole 0x7d51e
	 Extra: 

In [110]:
# Test node prediction
ram_data = np.fromfile("../../Videogame-Retrieval/ram_snes9x.bin", dtype="int16")
s = abstractify_state(ram_data, global_pos=True)
map_state_to_node(s, all_posns)

'Colosseum_R1'

In [161]:
# Return the name of the node that Samus should visit next
def get_path_advice(state, annotated_path):
    if annotated_path is None:
        return None
    dist, node = map_state_to_node(state, all_posns)
    i = annotated_path.index(node, state.items)
    if i is None:
        return None
    if i == len(annotated_path) - 1:
        return None
    return annotated_path[i+1][0]

In [25]:
advice_fn = """
area_offsets = {
    [0] = {[0] = 3, [1] = 10},
    [1] = {[0] = 0, [1] = 29},
    [2] = {[0] = 31, [1] = 49},
    [3] = {[0] = 37, [1] = 0},
    [4] = {[0] = 28, [1] = 29},
    [5] = {[0] = 0, [1] = 10},
    [6] = {[0] = 0, [1] = -10},
    [7] = {[0] = 0, [1] = 0}
}
local px_addr = 0x000AF6
local py_addr = 0x000AFA
local camx_addr = 0x000911
local camy_addr = 0x000915
local area_addr = 0x00079F
local mapx_addr = 0x0007A1
local mapy_addr = 0x0007A3
local camx = memory.readword(camx_addr)
local camy = memory.readword(camy_addr)
local px = memory.readword(px_addr)
local py = memory.readword(py_addr)
local screen_x = px - camx
local screen_y = py - camy
local xrad = memory.readbyte(0x7E0AFE)
local yrad = memory.readbyte(0x7E0B00)
local area = memory.readbyte(area_addr)
local mapx = memory.readbyte(mapx_addr)
local mapy = memory.readbyte(mapy_addr)
local px_global = 16 * area_offsets[area][0] + 16 * mapx + (px / 16)
local py_global = 16 * area_offsets[area][1] + 16 * mapy + (py / 16)
"""
advice2 = """
local go_x = {gox}
local go_y = {goy}
local to_x = go_x - px_global
local to_y = go_y - py_global
--print("hey")
--print(px_global, py_global)
--print(go_x, go_y)
--print(to_x, to_y)
gui.line(screen_x, screen_y, screen_x + (to_x * 16), screen_y + (to_y * 16), 0xffffffff)
local dist = math.sqrt((to_x + to_y) * (to_x + to_y))
gui.text(screen_x, screen_y, tostring(math.floor(dist)), 0xffffffff)
"""
def mk_advice(x, y):
    return advice_fn + advice2.format(gox=x, goy=y)

In [26]:
while True:
    f = open("../../Videogame-Retrieval/snes9x_bridge.lua", "w")
    c = all_posns["Colosseum_R2"]
    f.write(mk_advice(c.x, c.y))
    f.close()
    time.sleep(1)

KeyboardInterrupt: 

In [134]:
import multiprocessing

In [142]:
prooms = parse_rooms("../encoding/dsl/rooms_for_alloy.txt")
pexits = parse_exits("../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(prooms, pexits)
drop_table = get_drop_table(design)
ro = 

In [146]:
from bidict import bidict

In [147]:
# Node Name <-> Index
node_index_mapping = bidict({})
for i,n in enumerate(all_posns):
    node_index_mapping[n] = i

In [163]:
def oracle(current_node_index, current_advice_index):
    apath = None
    while True:
        current_node = node_index_mapping.inverse[current_node_index.value]
        s = read_state()
        dist, node = map_state_to_node(s, all_posns)
        if dist < 5:
            current_node = node
            current_node_index.value = node_index_mapping[current_node]
        print(f"Currently At: {current_node}")
        advice_node = get_path_advice((current_node, s.items), apath)
        if advice_node is None:
            # Then compute the path
            ctx = Context(current_node, s.items, "Landing_Site_Ship", ItemSet(["Mother_Brain"]), room_name_mapping)
            #ctx = Context("Landing_Site_Ship", ItemSet([]), "Morph_Ball_Room_MB", ItemSet(["MB"]))
            path = ctx.solve_path()
            ipath = get_intervals(path, drop_table)
            apath = get_annotated_path(path, ipath)
        advice_node = get_path_advice(s, apath)
        if advice_node is not None:
            print(f"You should go to: {advice_node}")
            advice_index.value = node_index_mapping[advice_node]
        else:
            print("No advice!")
        time.sleep(1)

In [159]:
current_node_index = multiprocessing.Value("d", 0)
current_advice_index = multiprocessing.Value("d", 0)

In [165]:
# The real deal
p = multiprocessing.Process(target=oracle, args=(current_node_index, current_advice_index))
p.run()
while True:
    f = open("../../Videogame-Retrieval/snes9x_bridge.lua", "w")
    a = node_index_mapping.inverse[current_advice_index]
    c = all_posns[a]
    f.write(mk_advice(c.x, c.y))
    f.close()
    time.sleep(1)

Currently At: The_Precious_Room_L1


TypeError: __init__() missing 1 required positional argument: 'room_name_mapping'

In [ ]:
# Where have you been?
# Don't update the plan in certain circumstances